# Construction de la bdd filtrée 

In [1]:
import pandas as pd

In [2]:
## Chemin Éléna
#enedis_final=pd.read_csv("Données/enedis_final.csv")
#chemin jérémie
enedis_final = pd.read_csv("/Users/jeremiedarracq/Stat-app/enedis_final.csv",sep=",")

On ne garde que les données inférieure au 31 juillet :

In [3]:
enedis_final=enedis_final.loc[enedis_final["date"]<"2021-07-31"]

On ne conserve que les plages de puissance 3 à 6 kvA / 6-9 kva / 9-12kva 

In [4]:
enedis_final.groupby(["plage_de_puissance_souscrite"])["nb_points_soutirage"].sum().reset_index()

,plage_de_puissance_souscrite,nb_points_soutirage
0,P0: Total <= 36 kVA,327120123782
1,P1: ]0-3] kVA,15297211995
2,P1: ]0-6] kVA,62460154025
3,P1: ]0-9] kVA,1319614428
4,P2: ]3-6] kVA,129867441852
5,P3: ]6-9] kVA,75962610871
6,P4: ]9-12] kVA,28865683345
7,P5: ]12-15] kVA,4203540697
8,P6: ]15-18] kVA,7330414029
9,P6: ]15-36] kVA,35481533


Donc on conserve cette plage de puissance couplé avec les profils RES1 (+ RES1WE) et 'RES11(+ RES11WE)'

In [5]:
enedis_filtered_3kva=enedis_final[(enedis_final["plage_de_puissance_souscrite"]=="P2: ]3-6] kVA")]
enedis_filtered_6kva=enedis_final[(enedis_final["plage_de_puissance_souscrite"]=="P3: ]6-9] kVA")&(enedis_final["profil"]=="RES11 (+ RES11WE)")]
enedis_filtered_9kva=enedis_final[(enedis_final["plage_de_puissance_souscrite"]=="P4: ]9-12] kVA")& (enedis_final["profil"]=="RES11 (+ RES11WE)")]


In [58]:
## On garde les données de température
temperature_3kva=enedis_filtered_3kva[["date","region_x","tmoy"]]
temperature_6kva=enedis_filtered_6kva[["date","region_x","tmoy"]]
temperature_9kva =enedis_filtered_9kva[["date","region_x","tmoy"]]

On importe la bdd :

In [7]:
## Chemine Jérémie
enedis_filtered_3kva.to_csv("/Users/jeremiedarracq/Stat-app/enedis_filtered_3kva.csv",index=False)
enedis_filtered_6kva.to_csv("/Users/jeremiedarracq/Stat-app/enedis_filtered_6kva.csv",index=False)
enedis_filtered_9kva.to_csv("/Users/jeremiedarracq/Stat-app/enedis_filtered_9kva.csv",index=False)
## Chemin Eléna 
#enedis_filtered_3kva.to_csv("Données/enedis_filtered_3kva.csv",index=False)
#enedis_filtered_6kva.to_csv("Données/enedis_filtered_6kva.csv",index=False)
#enedis_filtered_9kva.to_csv("Données/enedis_filtered_9kva.csv",index=False)


# Création des bdd comportant la variable conso_moyenne

Dans ce notebook, nous allons créer les tables contenant la variable consommation moyenne suivante : $conso\_moyenne=\frac{conso\_totale}{nb\_point\_soutirage}$

Observations : 
-  Chaques régions possède 3 groupements de points de soutirages classé par plage de puissance ( 3-6kva/6-9kva/9-12kva )
- Chaque groupement de points de soutirages possède un nombre de points de soutirages qui varié légèrement toutes les 30min (pose problème pour calculer la consommation moyenne)

Méthode : Nous allons attribuer un "indice" à chaque groupement de soutirage pour pouvoir l'identifier puis nous effectuerons une moyenne pour obtenir le nombre de point de soutirage moyen par groupement.

ex : 
- 20/06/2020 00:00 Bretagne 648983 points de soutirage
- 20/06/2020 08:00 Bretagne 648933 points de soutirage

 

On va attribuer un indice à chaque région afin de trouver la moyenne du nombre de points de soutirage par région :

In [8]:
enedis_filtered_3kva["indice_groupe"]=0
enedis_filtered_6kva["indice_groupe"]=0
enedis_filtered_9kva["indice_groupe"]=0

<ipython-input-8-457f8cf30308>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enedis_filtered_3kva["indice_groupe"]=0
<ipython-input-8-457f8cf30308>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enedis_filtered_6kva["indice_groupe"]=0
<ipython-input-8-457f8cf30308>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [10]:
temperature=enedis_filtered_3kva[["date","region_x","tmoy"]]
temperature=enedis_filtered_6kva[["date","region_x","tmoy"]]
temperature=enedis_filtered_9kva[["date","region_x","tmoy"]]

### Construction de la liste de référence du nombre de points de soutirage par groupement

On visualise la variation du nombre de points de soutirage par groupement sur une journée. On peut aussi remarqué que le nombre d'observations sur la journée diffère selon les groupements de points de soutirage.

In [11]:
france_point_soutirage_3kva =enedis_filtered_3kva[(enedis_filtered_3kva.date=="2020-06-20")].groupby(["region_x","nb_points_soutirage"])["horodate"].count().reset_index(name="count_apparition")
france_point_soutirage_3kva["id_soutirage"]=1
france_point_soutirage_3kva.sort_values("nb_points_soutirage",inplace=True)
france_point_soutirage_3kva = france_point_soutirage_3kva.reset_index(drop = True)
france_point_soutirage_3kva

,region_x,nb_points_soutirage,count_apparition,id_soutirage
0,Centre-Val de Loire,409567,2,1
1,Centre-Val de Loire,409619,20,1
2,Bourgogne-Franche-Comté,550060,3,1
3,Bourgogne-Franche-Comté,550133,24,1
4,Normandie,602675,1,1
5,Normandie,602736,21,1
6,Bretagne,648933,2,1
7,Bretagne,648983,14,1
8,Pays de la Loire,703823,3,1
9,Pays de la Loire,703886,19,1


In [18]:
france_point_soutirage_6kva =enedis_filtered_6kva[(enedis_filtered_6kva.date=="2020-06-20")].groupby(["region_x","nb_points_soutirage"])["horodate"].count().reset_index(name="count_apparition")
france_point_soutirage_6kva["id_soutirage"]=1
france_point_soutirage_6kva.sort_values("nb_points_soutirage",inplace=True)
france_point_soutirage_6kva = france_point_soutirage_6kva.reset_index(drop = True)
france_point_soutirage_6kva


,region_x,nb_points_soutirage,count_apparition,id_soutirage
0,Centre-Val de Loire,83131,2,1
1,Centre-Val de Loire,83139,24,1
2,Bourgogne-Franche-Comté,94191,2,1
3,Bourgogne-Franche-Comté,94194,20,1
4,Normandie,98123,3,1
5,Normandie,98125,20,1
6,Grand-Est,110604,22,1
7,Grand-Est,110605,1,1
8,Hauts-de-France,113704,3,1
9,Hauts-de-France,113705,26,1


In [19]:
france_point_soutirage_9kva =enedis_filtered_9kva[(enedis_filtered_9kva.date=="2020-06-20")].groupby(["region_x","nb_points_soutirage"])["horodate"].count().reset_index(name="count_apparition")
france_point_soutirage_9kva["id_soutirage"]=1
france_point_soutirage_9kva.sort_values("nb_points_soutirage",inplace=True)
france_point_soutirage_9kva = france_point_soutirage_9kva.reset_index(drop = True)
france_point_soutirage_9kva

,region_x,nb_points_soutirage,count_apparition,id_soutirage
0,Centre-Val de Loire,27162,20,1
1,Centre-Val de Loire,27163,1,1
2,Hauts-de-France,27406,27,1
3,Hauts-de-France,27407,3,1
4,Normandie,27968,20,1
5,Bourgogne-Franche-Comté,28833,20,1
6,Bourgogne-Franche-Comté,28834,1,1
7,Bretagne,32318,2,1
8,Bretagne,32319,19,1
9,Pays de la Loire,34932,24,1


In [14]:
def plage(value,epsilon,ref):
    ''' 
    Fonction qui permet de savoir si la valeur est bien dans la plage [ref - epsilon ; ref + epsilon]  '''
    if (ref-epsilon<=value<=ref+epsilon):
        return True

On crée nos listes de références

In [16]:
## Pour la plage 3kva
groupe_indice_3kva=1
region_ref = france_point_soutirage_3kva["region_x"][0]
list_ref_3kva =[[france_point_soutirage_3kva["region_x"][0],france_point_soutirage_3kva["nb_points_soutirage"][0]]]
for i in range(1,len(france_point_soutirage_3kva["region_x"])):
    if(france_point_soutirage_3kva["region_x"][i] != region_ref):
        groupe_indice_3kva+=1
        france_point_soutirage_3kva["id_soutirage"][i]=groupe_indice_3kva
        list_ref_3kva.append([france_point_soutirage_3kva["region_x"][i],france_point_soutirage_3kva["nb_points_soutirage"][i]])
        region_ref = france_point_soutirage_3kva["region_x"][i]
    else :
        france_point_soutirage_3kva["id_soutirage"][i]=groupe_indice_3kva
    
list_ref_3kva

<ipython-input-16-7281ff08be0a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point_soutirage_3kva["id_soutirage"][i]=groupe_indice_3kva
<ipython-input-16-7281ff08be0a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point_soutirage_3kva["id_soutirage"][i]=groupe_indice_3kva


[['Centre-Val de Loire', 409567],
 ['Bourgogne-Franche-Comté', 550060],
 ['Normandie', 602675],
 ['Bretagne', 648933],
 ['Pays de la Loire', 703823],
 ['Grand-Est', 815960],
 ['Nouvelle Aquitaine', 996617],
 ["Provence-Alpes-Côte d'Azur", 997352],
 ['Hauts-de-France', 1101532],
 ['Occitanie', 1142698],
 ['Auvergne-Rhône-Alpes', 1686425],
 ['Île-de-France', 3099647]]

In [20]:
## Pour la plage 6kva
groupe_indice_6kva=1
region_ref = france_point_soutirage_6kva["region_x"][0]
list_ref_6kva =[[france_point_soutirage_6kva["region_x"][0],france_point_soutirage_6kva["nb_points_soutirage"][0]]]
for i in range(1,len(france_point_soutirage_6kva["region_x"])):
    if(france_point_soutirage_6kva["region_x"][i] != region_ref):
        groupe_indice_6kva+=1
        france_point_soutirage_6kva["id_soutirage"][i]=groupe_indice_6kva
        list_ref_6kva.append([france_point_soutirage_6kva["region_x"][i],france_point_soutirage_6kva["nb_points_soutirage"][i]])
        region_ref = france_point_soutirage_6kva["region_x"][i]
    else :
        france_point_soutirage_6kva["id_soutirage"][i]=groupe_indice_6kva
    
list_ref_6kva

<ipython-input-20-1493fc11b121>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point_soutirage_6kva["id_soutirage"][i]=groupe_indice_6kva
<ipython-input-20-1493fc11b121>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point_soutirage_6kva["id_soutirage"][i]=groupe_indice_6kva


[['Centre-Val de Loire', 83131],
 ['Bourgogne-Franche-Comté', 94191],
 ['Normandie', 98123],
 ['Grand-Est', 110604],
 ['Hauts-de-France', 113704],
 ['Bretagne', 124361],
 ['Pays de la Loire', 135774],
 ["Provence-Alpes-Côte d'Azur", 173233],
 ['Occitanie', 213761],
 ['Nouvelle Aquitaine', 230678],
 ['Auvergne-Rhône-Alpes', 324914],
 ['Île-de-France', 509196]]

In [21]:
## Pour la plage 9kva
groupe_indice_9kva=1
region_ref = france_point_soutirage_9kva["region_x"][0]
list_ref_9kva =[[france_point_soutirage_9kva["region_x"][0],france_point_soutirage_9kva["nb_points_soutirage"][0]]]
for i in range(1,len(france_point_soutirage_9kva["region_x"])):
    if(france_point_soutirage_9kva["region_x"][i] != region_ref):
        groupe_indice_9kva+=1
        france_point_soutirage_9kva["id_soutirage"][i]=groupe_indice_9kva
        list_ref_9kva.append([france_point_soutirage_9kva["region_x"][i],france_point_soutirage_9kva["nb_points_soutirage"][i]])
        region_ref = france_point_soutirage_9kva["region_x"][i]
    else :
        france_point_soutirage_9kva["id_soutirage"][i]=groupe_indice_9kva
    
list_ref_9kva

<ipython-input-21-7867cc72059a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point_soutirage_9kva["id_soutirage"][i]=groupe_indice_9kva
<ipython-input-21-7867cc72059a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point_soutirage_9kva["id_soutirage"][i]=groupe_indice_9kva


[['Centre-Val de Loire', 27162],
 ['Hauts-de-France', 27406],
 ['Normandie', 27968],
 ['Bourgogne-Franche-Comté', 28833],
 ['Bretagne', 32318],
 ['Pays de la Loire', 34932],
 ['Grand-Est', 34948],
 ["Provence-Alpes-Côte d'Azur", 45399],
 ['Occitanie', 57608],
 ['Île-de-France', 69414],
 ['Nouvelle Aquitaine', 69612],
 ['Auvergne-Rhône-Alpes', 92316]]

Fonction pour attribuer l'indice du point de soutirage à chaque ligne de notre bdd en se basant sur la liste de référence 


In [47]:
def attribution(value,region,epsilon,list_ref):
    for i in range(len(list_ref)):
        if (list_ref[i][0]==region) & (list_ref[i][1]-epsilon<=value<=list_ref[i][1]+epsilon):
            return str(i)

In [48]:
enedis_filtered_3kva["indice_groupe"]=enedis_filtered_3kva.apply(lambda x: attribution(x.nb_points_soutirage,x.region_x,50000,list_ref_3kva),axis=1)
enedis_filtered_6kva["indice_groupe"]=enedis_filtered_6kva.apply(lambda x: attribution(x.nb_points_soutirage,x.region_x,50000,list_ref_6kva),axis=1)
enedis_filtered_9kva["indice_groupe"]=enedis_filtered_9kva.apply(lambda x: attribution(x.nb_points_soutirage,x.region_x,50000,list_ref_9kva),axis=1)

<ipython-input-48-c0e11bd90187>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enedis_filtered_3kva["indice_groupe"]=enedis_filtered_3kva.apply(lambda x: attribution(x.nb_points_soutirage,x.region_x,50000,list_ref_3kva),axis=1)
<ipython-input-48-c0e11bd90187>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enedis_filtered_6kva["indice_groupe"]=enedis_filtered_6kva.apply(lambda x: attribution(x.nb_points_soutirage,x.region_x,50000,list_ref_6kva),axis=1)
<ipython-input-48-c0e11bd90187>:3: SettingWithCopyWa

### Construction de la table contenant les totaux des points de soutirage 
Moyenne du nombre de points de soutirage par sous groupes 

In [53]:
table_region_points_3kva=enedis_filtered_3kva.groupby("indice_groupe")["nb_points_soutirage"].mean().reset_index()
table_region_points_3kva


,indice_groupe,nb_points_soutirage
0,0,4.133621e+05
1,1,5.539751e+05
2,10,1.704426e+06
3,11,3.112946e+06
4,2,6.078987e+05
5,3,6.563934e+05
6,4,7.108850e+05
7,5,8.217888e+05
8,6,1.008873e+06
9,7,1.009626e+06


In [54]:
table_region_points_6kva=enedis_filtered_6kva.groupby("indice_groupe")["nb_points_soutirage"].mean().reset_index()
table_region_points_6kva

,indice_groupe,nb_points_soutirage
0,0,85868.426152
1,1,97971.654154
2,10,335556.610181
3,11,512076.272328
4,2,102286.867826
5,3,115306.833217
6,4,119250.539579
7,5,131134.946365
8,6,142360.864196
9,7,179445.575513


In [55]:
table_region_points_9kva=enedis_filtered_9kva.groupby("indice_groupe")["nb_points_soutirage"].mean().reset_index()
table_region_points_9kva

,indice_groupe,nb_points_soutirage
0,0,28768.558907
1,1,29530.872743
2,10,74382.403947
3,11,99220.800678
4,2,30046.462004
5,3,31029.859329
6,4,34735.365984
7,5,37455.916280
8,6,37669.285532
9,7,48136.234412


## Consommation moyenne pour la france 
Création de la variable consommation moyenne pour la france (conso/nb de point soutirage)

In [80]:
nb_pts_3kva + nb_pts_6kva + nb_pts_9kva

15741362.878152324

In [57]:
## Pour les 3 kva
nb_pts_3kva =table_region_points_3kva.nb_points_soutirage.sum()
conso_totale_3kva =enedis_filtered_3kva.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
conso_totale_3kva["conso_totale_moyenne"]=conso_totale_3kva.total_energie_soutiree_wh/nb_pts_3kva

##Pour les 6 kva
nb_pts_6kva =table_region_points_6kva.nb_points_soutirage.sum()
conso_totale_6kva =enedis_filtered_6kva.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
conso_totale_6kva["conso_totale_moyenne"]=conso_totale_6kva.total_energie_soutiree_wh/nb_pts_6kva

##Pour les 9 kva
nb_pts_9kva =table_region_points_9kva.nb_points_soutirage.sum()
conso_totale_9kva =enedis_filtered_9kva.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
conso_totale_9kva["conso_totale_moyenne"]=conso_totale_9kva.total_energie_soutiree_wh/nb_pts_9kva



On merge temperature :

In [59]:
temperature_moy_3kva=temperature_3kva.groupby("date")["tmoy"].mean().reset_index()
temperature_moy_6kva=temperature_6kva.groupby("date")["tmoy"].mean().reset_index()
temperature_moy_9kva=temperature_9kva.groupby("date")["tmoy"].mean().reset_index()

In [60]:
conso_france_3kva=conso_totale_3kva.merge(temperature_moy_3kva,on="date",how="right")
conso_france_6kva=conso_totale_6kva.merge(temperature_moy_6kva,on="date",how="right")
conso_france_9kva=conso_totale_9kva.merge(temperature_moy_9kva,on="date",how="right")

In [62]:
## Chemin Éléna 
#conso_france_3kva.to_csv("Données/data_conso_moyenne/conso_france_3kva.csv")
## Chemin jeremie 
conso_france_3kva.to_csv("/Users/jeremiedarracq/Stat-app/conso_france_3kva.csv")
conso_france_6kva.to_csv("/Users/jeremiedarracq/Stat-app/conso_france_6kva.csv")
conso_france_9kva.to_csv("/Users/jeremiedarracq/Stat-app/conso_france_9kva.csv")


## Consommation moyenne pour les régions :
On crée maintenant les tables de conso moyenne pour chaque région :

In [82]:
def creation_table_conso_moyenne_3kva(region,nom_court):
    
    enedis_region=enedis_filtered_3kva[enedis_filtered_3kva.region_x==region]

    indice_groupement=enedis_region.indice_groupe.values[0] 
    nb_total_points_soutirage_region=table_region_points_3kva[table_region_points_3kva.indice_groupe==indice_groupement]["nb_points_soutirage"].values[0]

    conso_totale_region=enedis_region.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
    conso_totale_region["conso_totale_moyenne"]=conso_totale_region.total_energie_soutiree_wh/nb_total_points_soutirage_region


    temperature_region=temperature_3kva[temperature_3kva.region_x==region]
    temperature_region.drop_duplicates(inplace=True)

    conso_region=conso_totale_region.merge(temperature_region,on="date",how="right")
    ## Chemin éléna
    #conso_region.to_csv("Données/data_conso_moyenne/data_conso_"+nom_court+".csv",index=False)
    ## Chemin Jérémie
    conso_region.to_csv("/Users/jeremiedarracq/Stat-app/data_conso_3kva"+nom_court+".csv",index=False)


In [83]:
creation_table_conso_moyenne_3kva("Île-de-France","IDF")
creation_table_conso_moyenne_3kva("Hauts-de-France","HDF")
creation_table_conso_moyenne_3kva("Provence-Alpes-Côte d'Azur","PACA")
creation_table_conso_moyenne_3kva("Centre-Val de Loire","centre_val_de_loire")
creation_table_conso_moyenne_3kva("Normandie","Normandie")
creation_table_conso_moyenne_3kva("Occitanie","Occitanie")
creation_table_conso_moyenne_3kva("Auvergne-Rhône-Alpes","Auvergne-Rhône-Alpes")
creation_table_conso_moyenne_3kva("Bourgogne-Franche-Comté","Bourgogne_franche_comte")
creation_table_conso_moyenne_3kva("Nouvelle Aquitaine","Nouvelle_Aquitaine")
creation_table_conso_moyenne_3kva("Bretagne","Bretagne")
creation_table_conso_moyenne_3kva("Grand-Est","grand_est")
creation_table_conso_moyenne_3kva("Pays de la Loire","Pays_Loire")

<ipython-input-82-7f405be534cb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temperature_region.drop_duplicates(inplace=True)


In [84]:
def creation_table_conso_moyenne_6kva(region,nom_court):
    
    enedis_region=enedis_filtered_6kva[enedis_filtered_6kva.region_x==region]

    indice_groupement=enedis_region.indice_groupe.values[0] 
    nb_total_points_soutirage_region=table_region_points_6kva[table_region_points_9kva.indice_groupe==indice_groupement]["nb_points_soutirage"].values[0]

    conso_totale_region=enedis_region.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
    conso_totale_region["conso_totale_moyenne"]=conso_totale_region.total_energie_soutiree_wh/nb_total_points_soutirage_region


    temperature_region=temperature_6kva[temperature_6kva.region_x==region]
    temperature_region.drop_duplicates(inplace=True)

    conso_region=conso_totale_region.merge(temperature_region,on="date",how="right")
    ## Chemin éléna
    #conso_region.to_csv("Données/data_conso_moyenne/data_conso_"+nom_court+".csv",index=False)
    ## Chemin Jérémie
    conso_region.to_csv("/Users/jeremiedarracq/Stat-app/data_conso_6kva"+nom_court+".csv",index=False)


In [85]:
creation_table_conso_moyenne_6kva("Île-de-France","IDF")
creation_table_conso_moyenne_6kva("Hauts-de-France","HDF")
creation_table_conso_moyenne_6kva("Provence-Alpes-Côte d'Azur","PACA")
creation_table_conso_moyenne_6kva("Centre-Val de Loire","centre_val_de_loire")
creation_table_conso_moyenne_6kva("Normandie","Normandie")
creation_table_conso_moyenne_6kva("Occitanie","Occitanie")
creation_table_conso_moyenne_6kva("Auvergne-Rhône-Alpes","Auvergne-Rhône-Alpes")
creation_table_conso_moyenne_6kva("Bourgogne-Franche-Comté","Bourgogne_franche_comte")
creation_table_conso_moyenne_6kva("Nouvelle Aquitaine","Nouvelle_Aquitaine")
creation_table_conso_moyenne_6kva("Bretagne","Bretagne")
creation_table_conso_moyenne_6kva("Grand-Est","grand_est")
creation_table_conso_moyenne_6kva("Pays de la Loire","Pays_Loire")

<ipython-input-84-d08211ee2f40>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temperature_region.drop_duplicates(inplace=True)


In [86]:
def creation_table_conso_moyenne_9kva(region,nom_court):
    
    enedis_region=enedis_filtered_9kva[enedis_filtered_9kva.region_x==region]

    indice_groupement=enedis_region.indice_groupe.values[0] 
    nb_total_points_soutirage_region=table_region_points_9kva[table_region_points_9kva.indice_groupe==indice_groupement]["nb_points_soutirage"].values[0]

    conso_totale_region=enedis_region.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
    conso_totale_region["conso_totale_moyenne"]=conso_totale_region.total_energie_soutiree_wh/nb_total_points_soutirage_region


    temperature_region=temperature_9kva[temperature_9kva.region_x==region]
    temperature_region.drop_duplicates(inplace=True)

    conso_region=conso_totale_region.merge(temperature_region,on="date",how="right")
    ## Chemin éléna
    #conso_region.to_csv("Données/data_conso_moyenne/data_conso_"+nom_court+".csv",index=False)
    ## Chemin Jérémie
    conso_region.to_csv("/Users/jeremiedarracq/Stat-app/data_conso_9kva"+nom_court+".csv",index=False)

In [87]:
creation_table_conso_moyenne_9kva("Île-de-France","IDF")
creation_table_conso_moyenne_9kva("Hauts-de-France","HDF")
creation_table_conso_moyenne_9kva("Provence-Alpes-Côte d'Azur","PACA")
creation_table_conso_moyenne_9kva("Centre-Val de Loire","centre_val_de_loire")
creation_table_conso_moyenne_9kva("Normandie","Normandie")
creation_table_conso_moyenne_9kva("Occitanie","Occitanie")
creation_table_conso_moyenne_9kva("Auvergne-Rhône-Alpes","Auvergne-Rhône-Alpes")
creation_table_conso_moyenne_9kva("Bourgogne-Franche-Comté","Bourgogne_franche_comte")
creation_table_conso_moyenne_9kva("Nouvelle Aquitaine","Nouvelle_Aquitaine")
creation_table_conso_moyenne_9kva("Bretagne","Bretagne")
creation_table_conso_moyenne_9kva("Grand-Est","grand_est")
creation_table_conso_moyenne_9kva("Pays de la Loire","Pays_Loire")

<ipython-input-86-ad8cc5d3e4e0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temperature_region.drop_duplicates(inplace=True)
